In [1]:
import numpy as np
import torch
import torch.nn as nn
import csv
import os
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import time

In [2]:
from setup_dataset import *
from setup_model import *
from setup_model_types import *

In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [4]:
data = chunk_dataset(match_list=[1],
                     start_frame = 1,
                     end_frame = 1030
                    )

Successfully loaded NPZ.
Do you want to use GPU (y/n)n
Selected CPU


C:\Users\alece\Desktop\UnB\tcc\rnn_enduro\2-pytorch_train\setup_dataset.py:139: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  X_train = np.array(X_train)
C:\Users\alece\Desktop\UnB\tcc\rnn_enduro\2-pytorch_train\setup_dataset.py:140: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  Y_train = np.array(Y_train)


In [5]:
data.padSequence()
original_shape = data.data.shape
data.data = data.data.reshape(-1, 1, 100, 100)

In [6]:
data.data.shape, data.targets.shape, data.seq_len.shape

(torch.Size([3720, 1, 100, 100]), torch.Size([124, 30, 9]), torch.Size([124]))

In [7]:
device = data.device
input_size = data.data.shape[2]
output_size = len(data.available_targets)
match_list = data.match_list
start_frame = data.start_frame
end_frame = data.end_frame

In [8]:
class CNNLSTMModel1(nn.Module):
    def __init__(self, device, original_shape, output_size, hidden_dim, n_layers):
        super(CNNLSTMModel1, self).__init__()
        
        self.device = device
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        self.conv1 = nn.Conv2d(in_channels=1, out_channels=3, kernel_size=(3,3))
        self.conv2 = nn.Conv2d(3, 3, (10,10))
        self.activation = nn.ReLU()
        self.bnorm = nn.BatchNorm2d(num_features=3)
        self.pool = nn.MaxPool2d(kernel_size=(2,2))

        # output = (input - filter + 1) / stride
        # convolução 1: (100 - 3 + 1) / 1 = 98x98
        # pooling 1: 49x49
        # convolução 2: (45 - 10 + 1) / 1 = 36x36
        # pooling 2: 18x18
        # 18 * 18 * 3
        self.init_hidden(49*49*3)
        self.lstm = nn.LSTM(49*49*3, hidden_dim, n_layers, batch_first=True)  
        self.fc = nn.Linear(hidden_dim, output_size)
        self.out = nn.Softmax()
    
    def forward(self, x):
        
        hidden = self.init_hidden(49*49*3)

        x = self.pool(self.bnorm(self.activation(self.conv1(x))))
        # x = self.pool(self.bnorm(self.activation(self.conv2(x))))

        x = x.reshape(original_shape[0], original_shape[1], -1)
        
        x = pack_padded_sequence(x, data.seq_len, batch_first=True, enforce_sorted=False)
        
        pad_embed_pack_lstm = self.lstm(x, hidden)
        pad_embed_pack_lstm_pad = pad_packed_sequence(pad_embed_pack_lstm[0], batch_first=True)
        
        outs, _ = pad_embed_pack_lstm_pad
        
        out = outs.contiguous().view(-1, self.hidden_dim)
        out = self.fc(out)
        out = self.out(out)
        
        return out
        
    def init_hidden(self, input_size):
        # the weights are of the form (nb_layers, batch_size, nb_lstm_units)
        hidden_a = torch.randn(1, input_size, self.hidden_dim)
        hidden_b = torch.randn(1, input_size, self.hidden_dim)

        if self.device.type == 'cuda':
            hidden_a = hidden_a.cuda()
            hidden_b = hidden_b.cuda()

        hidden_a = Variable(hidden_a)
        hidden_b = Variable(hidden_b)

        return (hidden_a, hidden_b)

In [9]:
model_structure = ModelStructure(device, input_size, output_size, match_list, start_frame, end_frame)

Number of hidden neurons: 100
Number of epochs: 5000
Choose type of RNN model:
1 - Simple RNN
2 - LSTM
3 - CNN
type: 3
write a observations without space and punctuations:novo
models/CNN_novo_m1_f1to1030_epoch5000_H100 created


In [10]:
model = CNNLSTMModel1(device=model_structure.device,
                      original_shape=original_shape,
                      output_size=model_structure.output_size, 
                      hidden_dim=model_structure.hidden_neurons, 
                      n_layers=1)

In [11]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters())

In [12]:
if device.type == 'cuda':
    mymodel = model.cuda()
    X_train = data.data.cuda() 
    Y_train = data.targets.cuda()
else:
    mymodel = model
    X_train = data.data 
    Y_train = data.targets

In [13]:
device.type

'cpu'

In [14]:
def get_acc(predicted, target):
    
    predicted = torch.argmax(predicted, axis=1)
    target = torch.argmax(target, axis=1)

    acc = np.mean(predicted == target)
    
    return float(acc)

In [15]:
train_loss_arr = np.array([])
train_acc_arr = np.array([])

In [16]:
start_time_processing = time.time()

loss_file = open(model_structure.path + '/' + "loss_file.txt", "w")
first_time = True

best_loss = 1
first_epoch = True

for epoch in range(1, model_structure.n_epochs + 1):

    mymodel.train()

    optimizer.zero_grad()
    output = mymodel(X_train)
    loss = criterion(output, Y_train.view(-1,data.target_size[2]).float())
    loss.backward()
    optimizer.step()
        
    if epoch%10 == 0:

        train_loss_arr = np.append(train_loss_arr, loss.item())
        train_acc_arr  = np.append(train_acc_arr, model_structure.get_acc(output, Y_train.reshape(-1, len(data.available_targets))))
        
        loss_file.write("Epoch: {}/{}-------------------------------------------\n".format(epoch, model_structure.n_epochs))
        loss_file.write("Train -> Loss: {:.15f} Acc: {:.15f}\n".format(train_loss_arr[-1], train_acc_arr[-1]))
            
        print("Epoch: {}/{}-------------------------------------------".format(epoch, model_structure.n_epochs))
        print("Train -> Loss: {:.15f} Acc: {:.15f}".format(train_loss_arr[-1], train_acc_arr[-1]))
        
        if train_loss_arr[-1] < best_loss:
            state = { 'epoch': epoch + 1, 'state_dict': mymodel.state_dict(),
                      'optimizer': optimizer.state_dict(), 'losslogger': loss.item(), }
            torch.save(state, model_structure.path + '/' + model_structure.name)
            best_loss = loss.item()
        else:
            print("model not saved")
            
loss_file.write("--- %s seconds ---" % (time.time() - start_time_processing))
loss_file.close()
np.savez(model_structure.path + '/' + "train_loss_arr", train_loss_arr)
print("--- %s seconds ---" % (time.time() - start_time_processing))

C:\Users\alece\miniconda3\envs\torch\lib\site-packages\ipykernel_launcher.py:44: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch: 10/5000-------------------------------------------
Train -> Loss: 0.026716856285930 Acc: 0.175268813967705
Epoch: 20/5000-------------------------------------------
Train -> Loss: 0.024082902818918 Acc: 0.187365591526031
Epoch: 30/5000-------------------------------------------
Train -> Loss: 0.022238295525312 Acc: 0.200000002980232
Epoch: 40/5000-------------------------------------------
Train -> Loss: 0.020646858960390 Acc: 0.208602145314217
Epoch: 50/5000-------------------------------------------
Train -> Loss: 0.019469782710075 Acc: 0.217204302549362
Epoch: 60/5000-------------------------------------------
Train -> Loss: 0.018118679523468 Acc: 0.225268810987473
Epoch: 70/5000-------------------------------------------
Train -> Loss: 0.016861382871866 Acc: 0.233064517378807
Epoch: 80/5000-------------------------------------------
Train -> Loss: 0.015874003991485 Acc: 0.238172039389610
Epoch: 90/5000-------------------------------------------
Train -> Loss: 0.0151195507496

Epoch: 710/5000-------------------------------------------
Train -> Loss: 0.009005390107632 Acc: 0.273655921220779
Epoch: 720/5000-------------------------------------------
Train -> Loss: 0.009026864543557 Acc: 0.273655921220779
model not saved
Epoch: 730/5000-------------------------------------------
Train -> Loss: 0.009026085957885 Acc: 0.273655921220779
model not saved
Epoch: 740/5000-------------------------------------------
Train -> Loss: 0.009011184796691 Acc: 0.273655921220779
model not saved
Epoch: 750/5000-------------------------------------------
Train -> Loss: 0.009009875357151 Acc: 0.273655921220779
model not saved
Epoch: 760/5000-------------------------------------------
Train -> Loss: 0.009006401523948 Acc: 0.273655921220779
model not saved
Epoch: 770/5000-------------------------------------------
Train -> Loss: 0.009010874666274 Acc: 0.273655921220779
model not saved
Epoch: 780/5000-------------------------------------------
Train -> Loss: 0.009006990119815 Acc: 0.

Epoch: 1360/5000-------------------------------------------
Train -> Loss: 0.008975832723081 Acc: 0.273655921220779
model not saved
Epoch: 1370/5000-------------------------------------------
Train -> Loss: 0.008976116776466 Acc: 0.273655921220779
model not saved
Epoch: 1380/5000-------------------------------------------
Train -> Loss: 0.008975602686405 Acc: 0.273655921220779
model not saved
Epoch: 1390/5000-------------------------------------------
Train -> Loss: 0.008974694646895 Acc: 0.273655921220779
model not saved
Epoch: 1400/5000-------------------------------------------
Train -> Loss: 0.008974928408861 Acc: 0.273655921220779
model not saved
Epoch: 1410/5000-------------------------------------------
Train -> Loss: 0.008977248333395 Acc: 0.273655921220779
model not saved
Epoch: 1420/5000-------------------------------------------
Train -> Loss: 0.008976770564914 Acc: 0.273655921220779
model not saved
Epoch: 1430/5000-------------------------------------------
Train -> Loss: 0

Epoch: 2000/5000-------------------------------------------
Train -> Loss: 0.008971758186817 Acc: 0.273655921220779
model not saved
Epoch: 2010/5000-------------------------------------------
Train -> Loss: 0.008973944932222 Acc: 0.273655921220779
model not saved
Epoch: 2020/5000-------------------------------------------
Train -> Loss: 0.008971093222499 Acc: 0.273655921220779
model not saved
Epoch: 2030/5000-------------------------------------------
Train -> Loss: 0.008970439434052 Acc: 0.273655921220779
Epoch: 2040/5000-------------------------------------------
Train -> Loss: 0.008971595205367 Acc: 0.273655921220779
model not saved
Epoch: 2050/5000-------------------------------------------
Train -> Loss: 0.008971404284239 Acc: 0.273655921220779
model not saved
Epoch: 2060/5000-------------------------------------------
Train -> Loss: 0.008972215466201 Acc: 0.273655921220779
model not saved
Epoch: 2070/5000-------------------------------------------
Train -> Loss: 0.008971858769655

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\alece\miniconda3\envs\torch\lib\site-packages\IPython\core\interactiveshell.py", line 3437, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-16-86db77fb4d08>", line 16, in <module>
    loss.backward()
  File "C:\Users\alece\miniconda3\envs\torch\lib\site-packages\torch\tensor.py", line 245, in backward
    torch.autograd.backward(self, gradient, retain_graph, create_graph, inputs=inputs)
  File "C:\Users\alece\miniconda3\envs\torch\lib\site-packages\torch\autograd\__init__.py", line 147, in backward
    allow_unreachable=True, accumulate_grad=True)  # allow_unreachable flag
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\alece\miniconda3\envs\torch\lib\site-packages\IPython\core\interactiveshell.py", line 2061, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' objec

TypeError: object of type 'NoneType' has no len()